# Classes

In [94]:
from dataclasses import dataclass
from typing import List

In [95]:
@dataclass(slots=True)
class Location:
    latitude: float
    longitude: float
    altitude: float
    
@dataclass(slots=True)
class Motor:
    rpm: int
    
    def ramp_to_target(self, rpm: int, time_delta: int) -> None:
        # TODO: Implement ramping
        self.rpm = rpm

@dataclass(slots=True)
class Car:
    location: Location
    motor: Motor
    throttle: float
    
    def get_distance_traveled(self, time_delta: int) -> float:
        return time_delta / 1000 # Hardcoded 1 m/s
    
@dataclass(slots=True)
class Environment:
    time: int
    car: Car
    waypoints: List[Location]
    
class Simulation:
    saved_states: List[Environment]
    environment: Environment

    def step(self, time_delta: int) -> None:    
        distance = self.environment.car.get_distance_traveled(time_delta)
        
        
        self.saved_states.append(self.environment)
        
        

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m

track_data = pd.read_csv("./sem_2023_us.csv")

track_data = track_data.rename(columns={
    "Metres above sea level": "Altitude"
})

track_data.head(10)

,Latitude,Longitude,Altitude
0,39.799168,-86.238014,222.3313
1,39.799173,-86.237999,222.3617
2,39.799179,-86.237985,222.3676
3,39.799186,-86.237972,222.3918
4,39.799193,-86.237960,222.3986
5,39.799200,-86.237946,222.4268
6,39.799207,-86.237932,222.4488
7,39.799214,-86.237918,222.4692
8,39.799221,-86.237905,222.4826
9,39.799228,-86.237890,222.5074


In [97]:
# Earth's radius in meters
EARTH_RADIUS = 6371000

def great_circle_distance(latitude1: float, longitude1: float, latitude2: float, longitude2: float, radius: float) -> float:
    """Compute the greater circle distance on a sphere using the Haversine formula"""
    
    phi1 = m.radians(latitude1)
    phi2 = m.radians(latitude2)

    delta_phi = m.radians(latitude2 - latitude1)
    delta_lambda = m.radians(longitude2 - longitude1)

    a = m.sin(delta_phi / 2.0) ** 2 + m.cos(phi1) * m.cos(phi2) * m.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * m.atan2(m.sqrt(a), m.sqrt(1 - a))

    meters = radius * c
    return meters

def distance_with_altitude(distance: float, altitude1: float, altitude2: float):
    return m.sqrt(distance ** 2 + (altitude2 - altitude1) ** 2)




In [98]:
track_data["PreviousLatitude"] = track_data["Latitude"].shift(1)
track_data["PreviousLongitude"] = track_data["Longitude"].shift(1)
track_data["PreviousAltitude"] = track_data["Altitude"].shift(1)

track_data["FlatDistance"] = track_data.apply(lambda row: great_circle_distance(row["Latitude"], row["Longitude"], row["PreviousLatitude"], row["PreviousLongitude"], EARTH_RADIUS), axis=1)
track_data["AltitudeDistance"] = track_data.apply(lambda row: row["PreviousAltitude"] - row["Altitude"], axis=1)
track_data["Distance"] = track_data.apply(lambda row: distance_with_altitude(row["FlatDistance"], row["Altitude"], row["PreviousAltitude"]), axis=1)

track_data.head(10)



,Latitude,Longitude,Altitude,PreviousLatitude,PreviousLongitude,PreviousAltitude,FlatDistance,AltitudeDistance,Distance
0,39.799168,-86.238014,222.3313,NaN,NaN,NaN,NaN,NaN,NaN
1,39.799173,-86.237999,222.3617,39.799168,-86.238014,222.3313,1.445678,-0.0304,1.445997
2,39.799179,-86.237985,222.3676,39.799173,-86.237999,222.3617,1.375121,-0.0059,1.375133
3,39.799186,-86.237972,222.3918,39.799179,-86.237985,222.3676,1.334003,-0.0242,1.334222
4,39.799193,-86.237960,222.3986,39.799186,-86.237972,222.3918,1.307455,-0.0068,1.307473
5,39.799200,-86.237946,222.4268,39.799193,-86.237960,222.3986,1.386252,-0.0282,1.386539
6,39.799207,-86.237932,222.4488,39.799200,-86.237946,222.4268,1.433784,-0.0220,1.433953
7,39.799214,-86.237918,222.4692,39.799207,-86.237932,222.4488,1.422830,-0.0204,1.422976
8,39.799221,-86.237905,222.4826,39.799214,-86.237918,222.4692,1.386886,-0.0134,1.386951
9,39.799228,-86.237890,222.5074,39.799221,-86.237905,222.4826,1.437821,-0.0248,1.438035
